In [22]:
import pandas as pd 
import numpy as np
from Bio.Blast import NCBIWWW, NCBIXML
from Bio import SeqIO
from Bio.Blast.Applications import NcbiblastnCommandline
from io import StringIO
from Bio.Blast import NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
#please change the blastn_path to your local blastn engine. 
blastn_path = r"C:\Program Files\NCBI\blast-2.15.0+\bin\blastn.exe"
#This Blastref is required from Repbase (https://www.girinst.org/repbase/)
df_Blastref = pd.read_csv("./BLAST_ref_source/IR_search_reference.csv")

In [23]:
def Blast_csv(filename):
    df=pd.read_csv(filename)
    df['Seq_idA'] = df['Chr'] + ":" + df['repeat_start'].astype(str) + "-" + df['repeat_end'].astype(str)
    df2 = df[['Seq_idA']].copy().drop_duplicates(subset='Seq_idA', keep='first')
    merged_df = pd.merge(df_Blastref, df2, on='Seq_idA', how='inner')
    
    for index, row in merged_df.iterrows():
        sequence1 = row['SequencesA']
        sequence2 = row['SeqB_RC']
        seq1ID = row["Seq_idA"]
        seq2ID = row["Seq_idB"]

        seq1 = SeqRecord(Seq(sequence1),id=seq1ID)
        seq2 = SeqRecord(Seq(sequence2),id=seq2ID)
    
        SeqIO.write(seq1, "seq1.fasta", "fasta")
        SeqIO.write(seq2, "seq2.fasta", "fasta")
    
        blast_result = NcbiblastnCommandline(cmd=blastn_path,query="seq1.fasta", subject="seq2.fasta", strand="plus", reward=2, penalty=-3, gapopen=5, gapextend=2, word_size=11, outfmt=10)()[0]
        merged_df.at[index, 'BLAST_Result'] = blast_result
    
    merged_df['Top_Blast'] = merged_df['BLAST_Result'].str.split('\n').str[0]
    blast_columns = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
    merged_df[blast_columns] = merged_df['Top_Blast'].str.split(',', expand=True)
    BLASTed=merged_df.loc[merged_df['BLAST_Result'].str.len() > 5]
    
    df_BLASTed = pd.merge(df, BLASTed, on='Seq_idA', how='left', indicator=True)
    df_BLASTed['IR?'] = 'No'
    df_BLASTed.loc[df_BLASTed['_merge'] == 'both', 'IR?'] = 'Yes'
    df_BLASTed=df_BLASTed.drop(columns=['Seq_idA', 'SequencesA', 'SequencesB', 'Repeat_NameA', 'SeqB_RC', 'BLAST_Result', 'Top_Blast', 'qseqid', 'sseqid', 'mismatch','gapopen','qstart','qend','sstart','send','evalue','bitscore','_merge'])
    df_BLASTed = df_BLASTed.rename(columns={'Dist': 'IR_pair_distance', 'Seq_idB': 'IR_pair_Loc','Repeat_NameB':'IR_pair_Name','pident':'IR_pair_Pidentity','length':'IR_pair_matched_length'})
    
    df_BLASTed.to_csv(f"IR_BLASTed_{filename}",index=False)
    #return  merged_df